In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#Testing api key
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE-API-KEY"),
    temperature=0.3
)

response = llm.invoke("Say hello in Python")
print(response)

c:\Users\lenovo\miniconda3\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content='```python\nprint("Hello") \n```\n\n**Explanation:**\n\n* **`print()`:** This is a built-in Python function that displays output to the console.\n* **`"Hello"`:** This is a string literal. The text inside the double quotes is what will be displayed.\n\nWhen you run this code, it will simply output:\n\n```\nHello\n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--ced93bf6-8f71-4631-85c2-868771f7ecf9-0' usage_metadata={'input_tokens': 5, 'output_tokens': 80, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}}


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key = os.getenv("GOOGLE-API-KEY")
)

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text your job is to create {number} multiple choice questions in {tone} tone.
Make sure questions are not repeated and make sure that they follow the text above.
Make sure to format the response on basis of RESPONSE_JSON below.
##RESPONSE_JSON
{response_json} 
"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [8]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "2":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "3":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
}

In [27]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableSequence
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

quiz_chain = RunnableParallel({
    "quiz": quiz_generation_prompt | llm,
    "subject": RunnablePassthrough()
})

In [10]:
TEMPLATE2 = """
Youo are an expert english grammarian and writer. Given a multiple choice Quiz for {subject} students.
You need to evaluate the complexity of the quiz questions, and also check wheather is quiz is adapt to the quantitstive and cognitive ability of students.
If not yiu need to change the questions and change the tone so that it perfectly fits the students abilities.
QUIZ MCQ:{quiz}

Check as an expert english Writer for the above quiz.
"""

In [11]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [12]:
review_chain = RunnableSequence(
    quiz_evaluation_prompt,
    llm,
    RunnableLambda(lambda x: {"review": x})
)

In [13]:
from langchain_core.runnables import RunnableSequence

generate_evaluate_chain = quiz_chain | review_chain

In [14]:
file_path = "P:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\data.txt"
with open(file_path,"r") as file:
    TEXT = file.read()

In [15]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical viewpoint, probably approximately c

In [16]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice qustion", "options": {"a": "option 1", "b": "option 2", "c": "option 3", "d": "option 4"}, "answer": "correct answer"}, "2": {"mcq": "multiple choice qustion", "options": {"a": "option 1", "b": "option 2", "c": "option 3", "d": "option 4"}, "answer": "correct answer"}, "3": {"mcq": "multiple choice qustion", "options": {"a": "option 1", "b": "option 2", "c": "option 3", "d": "option 4"}, "answer": "correct answer"}}'

In [17]:
NUMBER = 5
SUBJECT = "python"
TONE = "simple"

In [18]:
from contextlib import contextmanager

@contextmanager
def get_gemini_callback():
    usage_log = {
        "prompt_tokens": 0,
        "completion_tokens": 0,
        "total_tokens": 0,
        "total_cost": 0.0
    }
    yield usage_log
    # You can log or return this after yield

In [28]:
quiz_output = quiz_chain.invoke(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

In [26]:
with get_gemini_callback() as gc:
    response = generate_evaluate_chain.invoke(
        {
            "text":TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [20]:
print(response)

{'review': AIMessage(content='Here\'s an evaluation of the provided quiz and suggestions for improvement, considering the target audience and the desired tone:\n\n## Quiz Evaluation:\n\n**Overall Complexity:** The quiz is generally well-structured and the questions directly relate to the provided text. The complexity of the questions is appropriate for an introductory understanding of the topic. The vocabulary used is largely accessible, aligning with a "simple" tone.\n\n**Quantitative and Cognitive Ability:**\n\n*   **Quantitative Ability:** The quiz does not heavily test quantitative ability, which is appropriate given the subject matter is primarily conceptual. There are no calculations or complex data interpretations required.\n*   **Cognitive Ability:** The questions assess recall of facts (who coined the term), understanding of definitions (main goal of ML), identification of subdisciplines, and comparison of related concepts (ML vs. Data Mining). This is suitable for testing com

In [23]:
response

{'review': AIMessage(content='This is a well-constructed quiz that effectively assesses comprehension of the provided text on Machine Learning. The questions are clear, directly related to the content, and the options are plausible distractors. The tone is appropriately simple and informative for a general audience or introductory learners.\n\nHere\'s an evaluation and breakdown:\n\n**Evaluation of Quiz Questions:**\n\n*   **Question 1: What is machine learning concerned with in artificial intelligence?**\n    *   **Complexity:** Simple. Directly tests the core definition.\n    *   **Cognitive Ability:** Recall and understanding of a key concept.\n    *   **Suitability:** Perfectly suitable.\n*   **Question 2: Who coined the term \'machine learning\'?**\n    *   **Complexity:** Simple. Tests a specific historical fact.\n    *   **Cognitive Ability:** Recall of a proper noun.\n    *   **Suitability:** Perfectly suitable.\n*   **Question 3: According to the text, what is a key difference

In [29]:
quiz_output

{'quiz': AIMessage(content='```json\n{\n  "1": {\n    "mcq": "Who coined the term \'machine learning\'?",\n    "options": {\n      "a": "Arthur Samuel",\n      "b": "Alan Turing",\n      "c": "Donald Hebb",\n      "d": "Geoffrey Hinton"\n    },\n    "answer": "Arthur Samuel"\n  },\n  "2": {\n    "mcq": "What is a subdiscipline of machine learning that has enabled neural networks to surpass previous approaches?",\n    "options": {\n      "a": "Data Mining",\n      "b": "Deep Learning",\n      "c": "Predictive Analytics",\n      "d": "Statistical Physics"\n    },\n    "answer": "Deep Learning"\n  },\n  "3": {\n    "mcq": "According to the text, what is a key difference between machine learning and data mining?",\n    "options": {\n      "a": "Machine learning focuses on discovery, while data mining focuses on prediction.",\n      "b": "Machine learning uses symbolic methods, while data mining uses statistical methods.",\n      "c": "Machine learning focuses on prediction, while data mini

In [30]:
res = quiz_output.get("quiz")
res

AIMessage(content='```json\n{\n  "1": {\n    "mcq": "Who coined the term \'machine learning\'?",\n    "options": {\n      "a": "Arthur Samuel",\n      "b": "Alan Turing",\n      "c": "Donald Hebb",\n      "d": "Geoffrey Hinton"\n    },\n    "answer": "Arthur Samuel"\n  },\n  "2": {\n    "mcq": "What is a subdiscipline of machine learning that has enabled neural networks to surpass previous approaches?",\n    "options": {\n      "a": "Data Mining",\n      "b": "Deep Learning",\n      "c": "Predictive Analytics",\n      "d": "Statistical Physics"\n    },\n    "answer": "Deep Learning"\n  },\n  "3": {\n    "mcq": "According to the text, what is a key difference between machine learning and data mining?",\n    "options": {\n      "a": "Machine learning focuses on discovery, while data mining focuses on prediction.",\n      "b": "Machine learning uses symbolic methods, while data mining uses statistical methods.",\n      "c": "Machine learning focuses on prediction, while data mining focuse

In [39]:
quiz_string = res.content
cleaned_string = quiz_string.strip("```json\n").strip("`")
final_mcqs = json.loads(cleaned_string)
final_mcqs
print(json.dumps(final_mcqs, indent=4))

{
    "1": {
        "mcq": "Who coined the term 'machine learning'?",
        "options": {
            "a": "Arthur Samuel",
            "b": "Alan Turing",
            "c": "Donald Hebb",
            "d": "Geoffrey Hinton"
        },
        "answer": "Arthur Samuel"
    },
    "2": {
        "mcq": "What is a subdiscipline of machine learning that has enabled neural networks to surpass previous approaches?",
        "options": {
            "a": "Data Mining",
            "b": "Deep Learning",
            "c": "Predictive Analytics",
            "d": "Statistical Physics"
        },
        "answer": "Deep Learning"
    },
    "3": {
        "mcq": "According to the text, what is a key difference between machine learning and data mining?",
        "options": {
            "a": "Machine learning focuses on discovery, while data mining focuses on prediction.",
            "b": "Machine learning uses symbolic methods, while data mining uses statistical methods.",
            "c": "Mach

In [40]:
quiz_table_data = []
for key,value in final_mcqs.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    answer = value["answer"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Answer":answer})


In [41]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Arthur Samuel | b: Alan Turing | c: Donald Hebb | d: Geoffrey Hinton',
  'Answer': 'Arthur Samuel'},
 {'MCQ': 'What is a subdiscipline of machine learning that has enabled neural networks to surpass previous approaches?',
  'Choices': 'a: Data Mining | b: Deep Learning | c: Predictive Analytics | d: Statistical Physics',
  'Answer': 'Deep Learning'},
 {'MCQ': 'According to the text, what is a key difference between machine learning and data mining?',
  'Choices': 'a: Machine learning focuses on discovery, while data mining focuses on prediction. | b: Machine learning uses symbolic methods, while data mining uses statistical methods. | c: Machine learning focuses on prediction, while data mining focuses on discovery of unknown properties. | d: Data mining is a subset of machine learning, while machine learning is not related to AI.',
  'Answer': 'Machine learning focuses on prediction, while data mining focuses on disc

In [43]:
quiz = pd.DataFrame(quiz_table_data)

In [44]:
quiz.to_csv("machinelearning.csv",index=False)

In [2]:
import pandas as pd
import os
import json
import traceback
from dotenv import load_dotenv
load_dotenv()

from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.looger import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage
from langchain.chains import LLMChain

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    google_api_key = os.getenv("GOOGLE-API-KEY"),
    temperature = 0.7,
)

quiz_generation_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "You are an expert MCQ quiz maker. You have to create the quiz based on user input and given instructions."
        ),
        HumanMessage(
            content = """
                Please create {number} muliple choice questions in a {tone} tone based on the following text.
                **Text:**
                {text}
                
                **Instructions:**
                1. Ensure that all questions are from given text only.
                2. Ensure that the questions are not repeated.
                3. Format your entire response as a JSON object that strictly follows below given scheme.
                {response_json}
            """
        )
    ]    
)

quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, ouput_key="quiz",verbose=True)

res = quiz_chain.invoke()

p:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lenovo\AppData\Local\Temp\ipykernel_23124\742896974.py:43: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, ouput_key="quiz",verbose=True)


ValidationError: 1 validation error for LLMChain
ouput_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='quiz', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden